In [54]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

### Class for the Neural Network

In [35]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [36]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [37]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [38]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Function for Local Search

In [75]:
def local_search_cv(num_iterations, initial_configuration, param_ranges, train_loader, test_loader , k_folds=5, use_scaling=True):
    best_accuracy = 0.0
    best_combination = initial_configuration
    current_configuration = initial_configuration
    train_times = []
    results = []
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    if use_scaling:
        scaler = StandardScaler()
        scaler.fit(train_loader.dataset.tensors[0].numpy())
        
    for _ in range(num_iterations):
        # Small changes to the current configuration
        new_configuration = {
            'Hidden Layer Sizes': [
                max(1, size + random.randint(-1, 1)) for size in current_configuration['Hidden Layer Sizes']
            ],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': max(param_ranges['min_lr'], min(param_ranges['max_lr'], current_configuration['Learning Rate'] + random.uniform(-0.01, 0.01))),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': max(1, current_configuration['Number of Epochs'] + random.randint(-1, 1))
        }
        fold_accuracies = []
        training_times = []

        for train_index, test_index in kf.split(train_loader.dataset):
            
            X_train, X_test = train_loader.dataset.tensors[0][train_index], train_loader.dataset.tensors[0][test_index]
            y_train, y_test = train_loader.dataset.tensors[1][train_index], train_loader.dataset.tensors[1][test_index]

            model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                       num_classes=num_classes,
                       hidden_layer_sizes=new_configuration['Hidden Layer Sizes'],
                       activation_function=new_configuration['Activation Function'])
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=new_configuration['Learning Rate'])

            if use_scaling:
                X_train_scaled = torch.tensor(scaler.transform(X_train.numpy()))
                X_test_scaled = torch.tensor(scaler.transform(X_test.numpy()))
            else:
                X_train_scaled, X_test_scaled = X_train, X_test
                 
            train_start_time = time.time()        
            train_model(model, DataLoader(TensorDataset(X_train_scaled, y_train), batch_size=new_configuration['Batch Size'], shuffle=True), optimizer, criterion, new_configuration['Number of Epochs'])
            train_end_time = time.time()
            train_time = train_end_time - train_start_time
          
            accuracy_test = check_accuracy(DataLoader(TensorDataset(X_test_scaled, y_test), batch_size=new_configuration['Batch Size'], shuffle=False), model)
            fold_accuracies.append(accuracy_test.item())
            training_times.append(train_time)

        avg_accuracy = np.mean(fold_accuracies)
        avg_train_time = np.mean(training_times)

        result_entry = {
                'Hidden Layer Sizes': new_configuration['Hidden Layer Sizes'],
                'Activation Function': new_configuration['Activation Function'].__name__,
                'Learning Rate': new_configuration['Learning Rate'],
                'Batch Size': new_configuration['Batch Size'],
                'Number of Epochs': new_configuration['Number of Epochs'],
                'Average Accuracy': accuracy_test.item(),
                'Average Training Time': avg_train_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = new_configuration
            current_configuration = new_configuration  # Update the current configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Loading / preparing data

### Wine Quality

In [41]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


### Congressional Voting

In [42]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


### Bank Marketing

In [43]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)
bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


### Testing Local search

In [79]:
dataset = cong_voting

train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

train_data = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

test_data = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

input_size = train_X.shape[1]

num_classes = 2

initial_configuration = {
    'Hidden Layer Sizes': [25, 30],
    'Activation Function': F.relu,
    'Learning Rate': 0.001,
    'Batch Size': 64,
    'Number of Epochs': 10
}

param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

num_iterations = 50

best_combination, best_accuracy, results_df_local = local_search_cv(
    num_iterations, initial_configuration, param_ranges, train_loader, test_loader, k_folds=5, use_scaling=True
)



Epoch 1/10, Average Loss: 0.6418403188387553
Epoch 2/10, Average Loss: 0.5725982387860616
Epoch 3/10, Average Loss: 0.5130886236826578
Epoch 4/10, Average Loss: 0.417840838432312
Epoch 5/10, Average Loss: 0.33574408292770386
Epoch 6/10, Average Loss: 0.2446001966794332
Epoch 7/10, Average Loss: 0.23068954547246298
Epoch 8/10, Average Loss: 0.1647811159491539
Epoch 9/10, Average Loss: 0.14967717975378036
Epoch 10/10, Average Loss: 0.11583720644315083
Epoch 1/10, Average Loss: 0.6322827339172363
Epoch 2/10, Average Loss: 0.6515175104141235
Epoch 3/10, Average Loss: 0.5417553981145223
Epoch 4/10, Average Loss: 0.5000475744406382
Epoch 5/10, Average Loss: 0.46386292576789856
Epoch 6/10, Average Loss: 0.35507388909657794
Epoch 7/10, Average Loss: 0.3079834779103597
Epoch 8/10, Average Loss: 0.24692516028881073
Epoch 9/10, Average Loss: 0.36126747727394104
Epoch 10/10, Average Loss: 0.23228796323140463
Epoch 1/10, Average Loss: 0.6185530225435892
Epoch 2/10, Average Loss: 0.5868179202079773


Epoch 4/11, Average Loss: 0.311258502304554
Epoch 5/11, Average Loss: 0.21630912274122238
Epoch 6/11, Average Loss: 0.18227268010377884
Epoch 7/11, Average Loss: 0.09444451704621315
Epoch 8/11, Average Loss: 0.10859508812427521
Epoch 9/11, Average Loss: 0.10076608136296272
Epoch 10/11, Average Loss: 0.07158561609685421
Epoch 11/11, Average Loss: 0.20523077622056007
Epoch 1/9, Average Loss: 0.25502654910087585
Epoch 2/9, Average Loss: 0.2548317313194275
Epoch 3/9, Average Loss: 0.18637334803740183
Epoch 4/9, Average Loss: 0.16739106178283691
Epoch 5/9, Average Loss: 0.14794768889745077
Epoch 6/9, Average Loss: 0.177018940448761
Epoch 7/9, Average Loss: 0.11223727092146873
Epoch 8/9, Average Loss: 0.10451897606253624
Epoch 9/9, Average Loss: 0.09606563051541646
Epoch 1/9, Average Loss: 0.25019728640715283
Epoch 2/9, Average Loss: 0.2524410088857015
Epoch 3/9, Average Loss: 0.22776009639104208
Epoch 4/9, Average Loss: 0.19515518099069595
Epoch 5/9, Average Loss: 0.19364224870999655
Epoch 

Epoch 5/9, Average Loss: 0.5240545630455017
Epoch 6/9, Average Loss: 0.494729083776474
Epoch 7/9, Average Loss: 0.44767196774482726
Epoch 8/9, Average Loss: 0.4150418519973755
Epoch 9/9, Average Loss: 0.36853439211845396
Epoch 1/9, Average Loss: 0.6807251930236816
Epoch 2/9, Average Loss: 0.6445205092430115
Epoch 3/9, Average Loss: 0.5824381232261657
Epoch 4/9, Average Loss: 0.5772867918014526
Epoch 5/9, Average Loss: 0.5336446523666382
Epoch 6/9, Average Loss: 0.5014700412750244
Epoch 7/9, Average Loss: 0.4644531011581421
Epoch 8/9, Average Loss: 0.4326706528663635
Epoch 9/9, Average Loss: 0.4031403064727783
Epoch 1/11, Average Loss: 0.2170539453625679
Epoch 2/11, Average Loss: 0.08552215788513422
Epoch 3/11, Average Loss: 0.01937112514860928
Epoch 4/11, Average Loss: 0.004780692188069225
Epoch 5/11, Average Loss: 0.003463871486019343
Epoch 6/11, Average Loss: 0.002846408938057721
Epoch 7/11, Average Loss: 0.0015188868310360704
Epoch 8/11, Average Loss: 0.0008730722789778156
Epoch 9/1

Epoch 6/9, Average Loss: 0.03116492092764626
Epoch 7/9, Average Loss: 0.020526130683720112
Epoch 8/9, Average Loss: 0.011842830572277308
Epoch 9/9, Average Loss: 0.0041560141544323415
Epoch 1/9, Average Loss: 0.6390724082787832
Epoch 2/9, Average Loss: 0.3430854876836141
Epoch 3/9, Average Loss: 0.16131709019343057
Epoch 4/9, Average Loss: 0.09396031747261684
Epoch 5/9, Average Loss: 0.06674919836223125
Epoch 6/9, Average Loss: 0.047383200687666736
Epoch 7/9, Average Loss: 0.03631627528617779
Epoch 8/9, Average Loss: 0.027513461808363598
Epoch 9/9, Average Loss: 0.020680734577278297
Epoch 1/9, Average Loss: 0.6229540506998698
Epoch 2/9, Average Loss: 0.23050335546334585
Epoch 3/9, Average Loss: 0.16059101124604544
Epoch 4/9, Average Loss: 0.19893126686414084
Epoch 5/9, Average Loss: 0.06929684927066167
Epoch 6/9, Average Loss: 0.04714841954410076
Epoch 7/9, Average Loss: 0.03520160727202892
Epoch 8/9, Average Loss: 0.024751704496641953
Epoch 9/9, Average Loss: 0.023069178064664204
Epoc

Epoch 1/11, Average Loss: 0.643992284933726
Epoch 2/11, Average Loss: 0.5679006377855936
Epoch 3/11, Average Loss: 0.4857747554779053
Epoch 4/11, Average Loss: 0.3971976637840271
Epoch 5/11, Average Loss: 0.3140096267064412
Epoch 6/11, Average Loss: 0.24973499278227487
Epoch 7/11, Average Loss: 0.22684710721174875
Epoch 8/11, Average Loss: 0.21361121535301208
Epoch 9/11, Average Loss: 0.12354044119517009
Epoch 10/11, Average Loss: 0.10294119641184807
Epoch 11/11, Average Loss: 0.107578178246816
Epoch 1/11, Average Loss: 0.603653222322464
Epoch 2/11, Average Loss: 0.10667602717876434
Epoch 3/11, Average Loss: 0.05736098648048937
Epoch 4/11, Average Loss: 0.04957795701920986
Epoch 5/11, Average Loss: 0.041159929474815726
Epoch 6/11, Average Loss: 0.021249646903015673
Epoch 7/11, Average Loss: 0.00884947250597179
Epoch 8/11, Average Loss: 0.001960574649274349
Epoch 9/11, Average Loss: 0.002252935490105301
Epoch 10/11, Average Loss: 0.0023280864115804434
Epoch 11/11, Average Loss: 0.000482

...

Epoch 2/11, Average Loss: 0.40485889216264087
Epoch 3/11, Average Loss: 0.19302185624837875
Epoch 4/11, Average Loss: 0.11357189590732257
Epoch 5/11, Average Loss: 0.0892857179666559
Epoch 6/11, Average Loss: 0.07487330213189125
Epoch 7/11, Average Loss: 0.055179310186455645
Epoch 8/11, Average Loss: 0.07887372250358264
Epoch 9/11, Average Loss: 0.03760207754870256
Epoch 10/11, Average Loss: 0.03125123027712107
Epoch 11/11, Average Loss: 0.028850893800457317
Epoch 1/11, Average Loss: 0.645776649316152
Epoch 2/11, Average Loss: 0.21545644849538803
Epoch 3/11, Average Loss: 0.18247819443543753
Epoch 4/11, Average Loss: 0.10112566749254863
Epoch 5/11, Average Loss: 0.07258463309456904
Epoch 6/11, Average Loss: 0.04921598872169852
Epoch 7/11, Average Loss: 0.04547088282803694
Epoch 8/11, Average Loss: 0.03505946012834708
Epoch 9/11, Average Loss: 0.02785447270919879
Epoch 10/11, Average Loss: 0.021850064241637785
Epoch 11/11, Average Loss: 0.020010560130079586
Epoch 1/11, Average Loss: 0.6

Epoch 8/9, Average Loss: 0.19846826791763306
Epoch 9/9, Average Loss: 0.21082069973150888
Epoch 1/9, Average Loss: 0.5862701833248138
Epoch 2/9, Average Loss: 0.6361892024676005
Epoch 3/9, Average Loss: 0.484441081682841
Epoch 4/9, Average Loss: 0.4354172746340434
Epoch 5/9, Average Loss: 0.37492695450782776
Epoch 6/9, Average Loss: 0.298432856798172
Epoch 7/9, Average Loss: 0.2585763136545817
Epoch 8/9, Average Loss: 0.23429089287916818
Epoch 9/9, Average Loss: 0.21705228090286255
Epoch 1/9, Average Loss: 0.6153314709663391
Epoch 2/9, Average Loss: 0.547665407260259
Epoch 3/9, Average Loss: 0.49889541665712994
Epoch 4/9, Average Loss: 0.4123940368493398
Epoch 5/9, Average Loss: 0.3369206289450328
Epoch 6/9, Average Loss: 0.26046858727931976
Epoch 7/9, Average Loss: 0.22107474009195963
Epoch 8/9, Average Loss: 0.16169475764036179
Epoch 9/9, Average Loss: 0.14851984630028406
Epoch 1/11, Average Loss: 0.26877259835600853
Epoch 2/11, Average Loss: 0.21895625442266464
Epoch 3/11, Average L

Epoch 1/9, Average Loss: 0.5907173156738281
Epoch 2/9, Average Loss: 0.5274318605661392
Epoch 3/9, Average Loss: 0.3958223760128021
Epoch 4/9, Average Loss: 0.3169725239276886
Epoch 5/9, Average Loss: 0.20254777371883392
Epoch 6/9, Average Loss: 0.35274727642536163
Epoch 7/9, Average Loss: 0.1213178038597107
Epoch 8/9, Average Loss: 0.11831242591142654
Epoch 9/9, Average Loss: 0.1610587127506733
Epoch 1/9, Average Loss: 0.5488260835409164
Epoch 2/9, Average Loss: 0.7603365778923035
Epoch 3/9, Average Loss: 0.4207181930541992
Epoch 4/9, Average Loss: 0.3480615168809891
Epoch 5/9, Average Loss: 0.32573357224464417
Epoch 6/9, Average Loss: 0.19668136537075043
Epoch 7/9, Average Loss: 0.18059106171131134
Epoch 8/9, Average Loss: 0.2118465080857277
Epoch 9/9, Average Loss: 0.16365677118301392
Epoch 1/9, Average Loss: 0.6239140629768372
Epoch 2/9, Average Loss: 0.5314004868268967
Epoch 3/9, Average Loss: 0.3987341523170471
Epoch 4/9, Average Loss: 0.40246187150478363
Epoch 5/9, Average Loss:

Epoch 3/9, Average Loss: 0.07530047670006752
Epoch 4/9, Average Loss: 0.09327122941613197
Epoch 5/9, Average Loss: 0.023378074634820223
Epoch 6/9, Average Loss: 0.01979756485670805
Epoch 7/9, Average Loss: 0.015772838587872685
Epoch 8/9, Average Loss: 0.007809716928750277
Epoch 9/9, Average Loss: 0.004317743261344731
Epoch 1/9, Average Loss: 0.7765839099884033
Epoch 2/9, Average Loss: 0.14017921872437
Epoch 3/9, Average Loss: 0.07100276164710521
Epoch 4/9, Average Loss: 0.03838279265910387
Epoch 5/9, Average Loss: 0.015453291218727827
Epoch 6/9, Average Loss: 0.009711636137217284
Epoch 7/9, Average Loss: 0.004599710035836324
Epoch 8/9, Average Loss: 0.003464768617413938
Epoch 9/9, Average Loss: 0.002437549876049161
Epoch 1/9, Average Loss: 0.6180257111787796
Epoch 2/9, Average Loss: 0.20051835626363754
Epoch 3/9, Average Loss: 0.05873409230262041
Epoch 4/9, Average Loss: 0.04973222110420465
Epoch 5/9, Average Loss: 0.02583763524889946
Epoch 6/9, Average Loss: 0.020234528044238687
Epoch

Epoch 3/10, Average Loss: 0.11018626764416695
Epoch 4/10, Average Loss: 0.07452744618058205
Epoch 5/10, Average Loss: 0.05625192075967789
Epoch 6/10, Average Loss: 0.04211312439292669
Epoch 7/10, Average Loss: 0.030932388734072447
Epoch 8/10, Average Loss: 0.029039460700005293
Epoch 9/10, Average Loss: 0.02744022337719798
Epoch 10/10, Average Loss: 0.03311003092676401
Epoch 1/10, Average Loss: 0.24977998435497284
Epoch 2/10, Average Loss: 0.11166258156299591
Epoch 3/10, Average Loss: 0.08342043310403824
Epoch 4/10, Average Loss: 0.05242032278329134
Epoch 5/10, Average Loss: 0.03644277574494481
Epoch 6/10, Average Loss: 0.08001760765910149
Epoch 7/10, Average Loss: 0.022928280057385564
Epoch 8/10, Average Loss: 0.021889036986976862
Epoch 9/10, Average Loss: 0.030293289572000504
Epoch 10/10, Average Loss: 0.013887748471461236
Epoch 1/10, Average Loss: 0.2312711551785469
Epoch 2/10, Average Loss: 0.22860898077487946
Epoch 3/10, Average Loss: 0.0846876185387373
Epoch 4/10, Average Loss: 0.

In [80]:
print("Best Combination:", best_combination)
print("Best Accuracy:", best_accuracy)

Best Combination: {'Hidden Layer Sizes': [24, 31], 'Activation Function': <function relu at 0x7fcf8ce0d550>, 'Learning Rate': 0.017848111481476588, 'Batch Size': 128, 'Number of Epochs': 10}
Best Accuracy: tensor(0.9706)


In [81]:
results_df_local

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average Training Time
0,"[25, 31]",sigmoid,0.008166,64,10,0.941176,0.059601
1,"[25, 30]",sigmoid,0.003806,64,9,0.941176,0.054432
2,"[25, 32]",relu,0.016306,128,10,0.941176,0.080669
3,"[24, 32]",relu,0.005713,128,11,0.911765,0.043033
4,"[25, 30]",tanh,0.001000,64,9,0.911765,0.061566
5,"[26, 32]",relu,0.012831,32,11,0.911765,0.087946
6,"[26, 30]",sigmoid,0.003627,128,10,0.558824,0.051189
7,"[24, 30]",sigmoid,0.002474,32,9,0.911765,0.090091
8,"[25, 30]",relu,0.011634,32,11,0.941176,0.111617
9,"[24, 31]",relu,0.017848,128,10,0.970588,0.038242


## Local search over all datasets

In [83]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing}

initial_configuration = {
    'Hidden Layer Sizes': [25, 30],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 32,
    'Number of Epochs': 10
}

param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

num_iterations = 50


all_local_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_combination, best_accuracy, results_df_local = local_search_cv(
    num_iterations, initial_configuration, param_ranges, train_loader, test_loader, k_folds=5, use_scaling=True)
    
    results_df_local['dataset'] = dataset_name
    all_local_results.append(results_df_local)

Epoch 1/11, Average Loss: 1.3000542127169095
Epoch 2/11, Average Loss: 1.0548241853713989
Epoch 3/11, Average Loss: 1.0336988247357881
Epoch 4/11, Average Loss: 1.0151475035227262
Epoch 5/11, Average Loss: 1.0019893719599797
Epoch 6/11, Average Loss: 0.9877028978787936
Epoch 7/11, Average Loss: 0.978136853988354
Epoch 8/11, Average Loss: 0.9629655874692477
Epoch 9/11, Average Loss: 0.955585176211137
Epoch 10/11, Average Loss: 0.9432551191403316
Epoch 11/11, Average Loss: 0.9320311839763935
Epoch 1/11, Average Loss: 1.2979093010608966
Epoch 2/11, Average Loss: 1.0652422831608699
Epoch 3/11, Average Loss: 1.046665555697221
Epoch 4/11, Average Loss: 1.0158042669296266
Epoch 5/11, Average Loss: 0.9930230727562538
Epoch 6/11, Average Loss: 0.9890644907951355
Epoch 7/11, Average Loss: 0.9782768790538494
Epoch 8/11, Average Loss: 0.9649244684439439
Epoch 9/11, Average Loss: 0.9538621031321012
Epoch 10/11, Average Loss: 0.9477408785086412
Epoch 11/11, Average Loss: 0.9319822980807377
Epoch 1/1

Epoch 9/11, Average Loss: 1.0117372366098256
Epoch 10/11, Average Loss: 1.0041352226183964
Epoch 11/11, Average Loss: 0.9905590052788075
Epoch 1/11, Average Loss: 1.1781496864098768
Epoch 2/11, Average Loss: 1.0989536239550663
Epoch 3/11, Average Loss: 1.0730336404763736
Epoch 4/11, Average Loss: 1.0488327769132761
Epoch 5/11, Average Loss: 1.0351719310650458
Epoch 6/11, Average Loss: 1.0273260332070864
Epoch 7/11, Average Loss: 1.0225254045082972
Epoch 8/11, Average Loss: 1.0120610819413112
Epoch 9/11, Average Loss: 1.0009970532013819
Epoch 10/11, Average Loss: 0.9907054502230425
Epoch 11/11, Average Loss: 0.9872238136254824
Epoch 1/11, Average Loss: 1.1636496433844934
Epoch 2/11, Average Loss: 1.0947461421673115
Epoch 3/11, Average Loss: 1.0621595712808463
Epoch 4/11, Average Loss: 1.0465739571131192
Epoch 5/11, Average Loss: 1.0417828353551717
Epoch 6/11, Average Loss: 1.0282855946284073
Epoch 7/11, Average Loss: 1.0223382633465987
Epoch 8/11, Average Loss: 1.007240240390484
Epoch 9

Epoch 8/10, Average Loss: 0.9674506783485413
Epoch 9/10, Average Loss: 0.9680264267054471
Epoch 10/10, Average Loss: 0.9660226612380056
Epoch 1/10, Average Loss: 1.2852119424126365
Epoch 2/10, Average Loss: 1.090639378085281
Epoch 3/10, Average Loss: 1.0309185313455986
Epoch 4/10, Average Loss: 1.0120009382565816
Epoch 5/10, Average Loss: 0.9952924739230763
Epoch 6/10, Average Loss: 0.9798620867006707
Epoch 7/10, Average Loss: 0.9715729626742277
Epoch 8/10, Average Loss: 0.9619619467041709
Epoch 9/10, Average Loss: 0.9578159541794748
Epoch 10/10, Average Loss: 0.9581326047579447
Epoch 1/10, Average Loss: 1.2128518998622895
Epoch 2/10, Average Loss: 1.0877585168068225
Epoch 3/10, Average Loss: 1.0630720427403084
Epoch 4/10, Average Loss: 1.0296851428655478
Epoch 5/10, Average Loss: 1.036291023859611
Epoch 6/10, Average Loss: 1.0313753091371975
Epoch 7/10, Average Loss: 1.0095722290185782
Epoch 8/10, Average Loss: 1.0146135403559757
Epoch 9/10, Average Loss: 0.9945146019642169
Epoch 10/1

Epoch 9/10, Average Loss: 1.000337768684734
Epoch 10/10, Average Loss: 0.9931397004560991
Epoch 1/10, Average Loss: 1.283268234946511
Epoch 2/10, Average Loss: 1.0881106076818523
Epoch 3/10, Average Loss: 1.055674337979519
Epoch 4/10, Average Loss: 1.0557904225407224
Epoch 5/10, Average Loss: 1.0343738234404363
Epoch 6/10, Average Loss: 1.0212690270308293
Epoch 7/10, Average Loss: 1.0110538746371414
Epoch 8/10, Average Loss: 1.005407822854591
Epoch 9/10, Average Loss: 0.9967270222577181
Epoch 10/10, Average Loss: 1.0066801941756047
Epoch 1/10, Average Loss: 1.2936076919237773
Epoch 2/10, Average Loss: 1.0813534386230237
Epoch 3/10, Average Loss: 1.0678221709800488
Epoch 4/10, Average Loss: 1.04500848235506
Epoch 5/10, Average Loss: 1.0269347483461553
Epoch 6/10, Average Loss: 1.0146734714508057
Epoch 7/10, Average Loss: 1.0129513144493103
Epoch 8/10, Average Loss: 1.0013955542535493
Epoch 9/10, Average Loss: 0.9973352804328456
Epoch 10/10, Average Loss: 0.9868923353426384
Epoch 1/11, A

Epoch 8/11, Average Loss: 1.0226579489129963
Epoch 9/11, Average Loss: 1.0118699597589897
Epoch 10/11, Average Loss: 1.0032118107333328
Epoch 11/11, Average Loss: 1.008929507298903
Epoch 1/11, Average Loss: 1.2210292635541973
Epoch 2/11, Average Loss: 1.095079739888509
Epoch 3/11, Average Loss: 1.0801380583734224
Epoch 4/11, Average Loss: 1.0626088254379504
Epoch 5/11, Average Loss: 1.0535953297759548
Epoch 6/11, Average Loss: 1.0427287130644827
Epoch 7/11, Average Loss: 1.0328836206233862
Epoch 8/11, Average Loss: 1.0285785360769792
Epoch 9/11, Average Loss: 1.0186864137649536
Epoch 10/11, Average Loss: 1.00871310450814
Epoch 11/11, Average Loss: 1.0063307483990986
Epoch 1/11, Average Loss: 1.2354816794395447
Epoch 2/11, Average Loss: 1.092422447421334
Epoch 3/11, Average Loss: 1.078941670331088
Epoch 4/11, Average Loss: 1.068457599842187
Epoch 5/11, Average Loss: 1.0619719642581362
Epoch 6/11, Average Loss: 1.0454029928554187
Epoch 7/11, Average Loss: 1.0399762681036284
Epoch 8/11, A

...

Epoch 7/10, Average Loss: 0.27990789906781854
Epoch 8/10, Average Loss: 0.2782818968724279
Epoch 9/10, Average Loss: 0.2783053626568572
Epoch 10/10, Average Loss: 0.27701471420457063
Epoch 1/10, Average Loss: 0.2946935330779807
Epoch 2/10, Average Loss: 0.2801403153723883
Epoch 3/10, Average Loss: 0.27790052892224304
Epoch 4/10, Average Loss: 0.2761420094344801
Epoch 5/10, Average Loss: 0.276088657056533
Epoch 6/10, Average Loss: 0.2744787595081098
Epoch 7/10, Average Loss: 0.2735065281825158
Epoch 8/10, Average Loss: 0.2726465562738261
Epoch 9/10, Average Loss: 0.27186938758613993
Epoch 10/10, Average Loss: 0.2711301156618063
Epoch 1/10, Average Loss: 0.297246589171655
Epoch 2/10, Average Loss: 0.28018304307773273
Epoch 3/10, Average Loss: 0.2794919493562967
Epoch 4/10, Average Loss: 0.27726332742033655
Epoch 5/10, Average Loss: 0.27695846615485775
Epoch 6/10, Average Loss: 0.2765647015959314
Epoch 7/10, Average Loss: 0.2748322761029873
Epoch 8/10, Average Loss: 0.2741967939491411
Epo

Epoch 6/9, Average Loss: 0.2803902156578684
Epoch 7/9, Average Loss: 0.27965438271899823
Epoch 8/9, Average Loss: 0.27848447692915074
Epoch 9/9, Average Loss: 0.2779557396051953
Epoch 1/9, Average Loss: 0.30131699354613867
Epoch 2/9, Average Loss: 0.28203590011712415
Epoch 3/9, Average Loss: 0.2784232222339482
Epoch 4/9, Average Loss: 0.2764190459280338
Epoch 5/9, Average Loss: 0.276782807649918
Epoch 6/9, Average Loss: 0.27522515954033844
Epoch 7/9, Average Loss: 0.27293125605120244
Epoch 8/9, Average Loss: 0.2719645725147238
Epoch 9/9, Average Loss: 0.27148055617959754
Epoch 1/9, Average Loss: 0.3005847280609955
Epoch 2/9, Average Loss: 0.28184338543310905
Epoch 3/9, Average Loss: 0.2785202138198232
Epoch 4/9, Average Loss: 0.277028853381143
Epoch 5/9, Average Loss: 0.2760759327307488
Epoch 6/9, Average Loss: 0.27431415723076147
Epoch 7/9, Average Loss: 0.27334694426904604
Epoch 8/9, Average Loss: 0.27209839763716587
Epoch 9/9, Average Loss: 0.27209258509926426
Epoch 1/9, Average Los

Epoch 3/10, Average Loss: 0.2784496077598588
Epoch 4/10, Average Loss: 0.27638418561986927
Epoch 5/10, Average Loss: 0.2754072320548076
Epoch 6/10, Average Loss: 0.2742374972847191
Epoch 7/10, Average Loss: 0.27307063400817727
Epoch 8/10, Average Loss: 0.2726578016593618
Epoch 9/10, Average Loss: 0.27038436102707986
Epoch 10/10, Average Loss: 0.27050649423550055
Epoch 1/10, Average Loss: 0.2902904723599119
Epoch 2/10, Average Loss: 0.28122869801246425
Epoch 3/10, Average Loss: 0.2787716839173176
Epoch 4/10, Average Loss: 0.27794524459439574
Epoch 5/10, Average Loss: 0.27618974230243165
Epoch 6/10, Average Loss: 0.2762008453674108
Epoch 7/10, Average Loss: 0.2751221422974056
Epoch 8/10, Average Loss: 0.2748019699441287
Epoch 9/10, Average Loss: 0.27376287801245464
Epoch 10/10, Average Loss: 0.2727016804432406
Epoch 1/10, Average Loss: 0.2914039434015172
Epoch 2/10, Average Loss: 0.2815062231700687
Epoch 3/10, Average Loss: 0.2792152299645167
Epoch 4/10, Average Loss: 0.27810282652600876

Epoch 5/11, Average Loss: 0.2862251177972671
Epoch 6/11, Average Loss: 0.28652277851061336
Epoch 7/11, Average Loss: 0.28531225342122674
Epoch 8/11, Average Loss: 0.28581942010273054
Epoch 9/11, Average Loss: 0.2886369941050856
Epoch 10/11, Average Loss: 0.2846434801563765
Epoch 11/11, Average Loss: 0.28399382063269035
Epoch 1/11, Average Loss: 0.2966122184021091
Epoch 2/11, Average Loss: 0.2849873766773245
Epoch 3/11, Average Loss: 0.2857925418438842
Epoch 4/11, Average Loss: 0.2815203149124835
Epoch 5/11, Average Loss: 0.28045348233534295
Epoch 6/11, Average Loss: 0.2823873446910705
Epoch 7/11, Average Loss: 0.2790935443153659
Epoch 8/11, Average Loss: 0.27875461579454175
Epoch 9/11, Average Loss: 0.28048193662707666
Epoch 10/11, Average Loss: 0.27938253998539403
Epoch 11/11, Average Loss: 0.2797218127338921
Epoch 1/11, Average Loss: 0.3004791545274767
Epoch 2/11, Average Loss: 0.288303682381667
Epoch 3/11, Average Loss: 0.2850988014121947
Epoch 4/11, Average Loss: 0.2833466180645436

Epoch 8/11, Average Loss: 0.27499587374549467
Epoch 9/11, Average Loss: 0.27472765951697686
Epoch 10/11, Average Loss: 0.27343550170726566
Epoch 11/11, Average Loss: 0.2732261928178153
Epoch 1/11, Average Loss: 0.2995099694223948
Epoch 2/11, Average Loss: 0.28190872360708064
Epoch 3/11, Average Loss: 0.27894247589108434
Epoch 4/11, Average Loss: 0.27771209798825597
Epoch 5/11, Average Loss: 0.27647050242256194
Epoch 6/11, Average Loss: 0.275264806973124
Epoch 7/11, Average Loss: 0.2745217307497865
Epoch 8/11, Average Loss: 0.2747187162342581
Epoch 9/11, Average Loss: 0.27283340673532014
Epoch 10/11, Average Loss: 0.27306766466246646
Epoch 11/11, Average Loss: 0.2718875849782263


In [84]:
all_local_results_df = pd.concat(all_local_results, ignore_index=True)
all_local_results_df = pd.DataFrame(all_local_results_df)


In [85]:
all_local_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average Training Time,dataset
0,"[25, 30]",tanh,0.014512,64,11,0.537055,1.192598,wine_quality
1,"[24, 29]",relu,0.022347,128,12,0.534167,0.866983,wine_quality
2,"[24, 31]",relu,0.020818,64,12,0.539942,1.399698,wine_quality
3,"[24, 31]",sigmoid,0.029520,32,11,0.547642,2.171049,wine_quality
4,"[25, 32]",relu,0.039251,32,10,0.536092,2.244884,wine_quality
...,...,...,...,...,...,...,...,...
145,"[26, 32]",relu,0.012418,64,9,0.890744,4.400872,bank_marketing
146,"[25, 30]",relu,0.021314,64,11,0.888467,5.353774,bank_marketing
147,"[25, 31]",sigmoid,0.014126,32,10,0.893778,7.776461,bank_marketing
148,"[27, 32]",tanh,0.025215,128,10,0.893627,3.617222,bank_marketing


In [86]:
all_local_results_df.to_csv('local_search_results.csv', index=False)

In [88]:
top_models_rows = []

for dataset in all_local_results_df['dataset'].unique():
    top_models_rows.extend(all_local_results_df[all_local_results_df['dataset'] == dataset].nlargest(2, 'Average Accuracy').iterrows())

top_models_rows_data = [row[1] for row in top_models_rows]

top_models_df = pd.DataFrame(top_models_rows_data).reset_index(drop=True)

top_models_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Average Training Time,dataset
0,"[24, 31]",relu,0.031303,128,10,0.562079,1.284953,wine_quality
1,"[24, 30]",tanh,0.023515,64,10,0.562079,0.822094,wine_quality
2,"[24, 32]",tanh,0.012187,64,12,0.970588,0.051132,cong_voting
3,"[25, 32]",tanh,0.020327,32,11,0.970588,0.064261,cong_voting
4,"[26, 31]",sigmoid,0.015217,32,10,0.896813,7.707599,bank_marketing
5,"[25, 31]",tanh,0.016443,128,9,0.896510,3.358007,bank_marketing
